In [ ]:
# File: yarn.ipynb
# Author: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
import numpy as np
from frontend import App


def make_strands(offset: float, width: float, res: float):

    offset = 1.3 * offset
    n_vertical_yarns = int(res * 0.28 / offset)
    n_points_per_seg = int(res * 20)
    n_segs = int(width * n_vertical_yarns)
    n_points = n_segs * n_points_per_seg
    dx = 1.0 / (n_points - 1)
    strands = []

    for k in range(n_vertical_yarns):
        y_base = k / (n_vertical_yarns - 1) - 0.5 if n_vertical_yarns > 1 else 0.0
        z_base = 0.0
        j_vals = np.arange(n_points)
        x_mod = (j_vals % n_points_per_seg) / n_points_per_seg
        t = 2.0 * np.pi * x_mod
        x_disp = -width * (0.5 / n_segs) * np.sin(2.0 * t)
        y_disp = (0.85 / n_vertical_yarns) * np.sin(t)
        z_disp = 0.75 * offset * np.cos(2.0 * t)
        x_coord = width * (2.0 * dx * j_vals - 1.0) + x_disp
        y_coord = y_base + y_disp
        z_coord = z_base + z_disp
        xyz = np.zeros((n_points, 3))
        xyz[:, 0] = x_coord
        xyz[:, 1] = y_coord
        xyz[:, 2] = z_coord
        strands.append((xyz, False))

    for pos_index in range(2):
        for k in range(n_segs - 1):
            dx_local = 2.0 * width / n_segs
            if pos_index == 0:
                y_base = 0.5 + 0.25 * dx_local
            else:
                y_base = -0.5 - 0.25 * dx_local
            z_base = 0.15 * dx_local
            x_center = dx_local * (k + 0.77) - width
            if pos_index == 1:
                x_center += 0.5 * dx_local
            j_vals = np.arange(n_points_per_seg)
            t = 2.0 * np.pi * j_vals / n_points_per_seg
            r = 0.78 * width / n_segs
            z_val = r * np.cos(t)
            theta = 0.25 * np.pi
            x_coord = x_center + r * np.sin(t)
            if pos_index == 0:
                y_coord = y_base + z_val * np.sin(theta)
            else:
                y_coord = y_base - z_val * np.sin(theta)
            z_coord = z_base + z_val * np.cos(theta)
            xyz = np.zeros((n_points_per_seg, 3))
            xyz[:, 0] = x_coord
            xyz[:, 1] = y_coord
            xyz[:, 2] = z_coord
            strands.append((xyz, True))

    return strands


app = App.create("yarn")
scene = app.scene.create()

k = 0
for V, closed in make_strands(4e-3, 0.5, 1.0):
    E = [[i, i + 1] for i in range(len(V) - 1)]
    if closed:
        E.append([len(V) - 1, 0])
    name = f"strand-{k}"
    app.asset.add.rod(name, V, np.array(E, dtype=np.uint32))
    obj = scene.add(name)
    (
        obj.param.set("bend", 0.0)
        .set("young-mod", 1e5)
        .set("contact-gap", 1e-3)
        .set("contact-offset", 2.3e-3)
        .set("length-factor", 0.85)
    )
    if not closed:
        move_delta, t_end = -5, 10
        obj.pin(obj.grab([-1, 0, 0])).move_by([move_delta, 0, 0], t_end)
        obj.pin(obj.grab([1, 0, 0])).move_by([-move_delta, 0, 0], t_end)
    k += 1

scene = scene.build().report()
scene.preview()

In [ ]:
session = app.session.create(scene)
session.param.set("frames", 120).set("dt", 1e-2).set("gravity", 0.0)
session = session.build()

In [ ]:
session.start().preview()
session.stream()

In [ ]:
session.animate()

In [ ]:
session.export.animation()

In [ ]:
# this is for CI
assert session.finished()